# Projet électif python: Prédictions et machine learning sur la NBA

In [33]:
# %pip install pandas numpy plotly requests scikit-learn

import pandas as pd
import numpy as np

Dans cette dernière partie, nous avons tenté une approche basée sur du machine learning afin de prédire l'issue de matchs futurs de NBA. Pour ce faire, nous avons commencé par une récolte des données par un scrapping web depuis sur site `https://www.basketball-reference.com`, puis nous avons nettoyé et filtré ces données pour en faire un `.csv` global contenant l'ensemble des données des matchs joués en saison régulière et playoffs des saisons 2015-16 à 2023-24.

#### Ouverure et nettoyage du dataset

In [34]:
df = pd.read_csv("nba_box_scores.csv", index_col=0)
df = df.sort_values("date")
df = df.reset_index(drop=True)
del df["mp.1"]
del df["mp_opp.1"]
del df["index_opp"]
df

,Unnamed: 0,mp,fg,fga,fg%,3p,3pa,3p%,ft,fta,...,tov%_max_opp,usg%_max_opp,ortg_max_opp,drtg_max_opp,team_opp,total_opp,home_opp,season,date,won
0,16087,240.0,41.0,96.0,0.427,9.0,30.0,0.300,20.0,22.0,...,37.5,38.9,201.0,120.0,NOP,95,0,2016,2015-10-27,True
1,16086,240.0,35.0,83.0,0.422,6.0,18.0,0.333,19.0,27.0,...,69.4,43.7,206.0,104.0,GSW,111,1,2016,2015-10-27,False
2,16905,240.0,37.0,87.0,0.425,7.0,19.0,0.368,16.0,23.0,...,30.4,29.0,138.0,105.0,CLE,95,0,2016,2015-10-27,True
3,16904,240.0,38.0,94.0,0.404,9.0,29.0,0.310,10.0,17.0,...,53.2,34.6,162.0,104.0,CHI,97,1,2016,2015-10-27,False
4,1225,240.0,37.0,82.0,0.451,8.0,27.0,0.296,12.0,15.0,...,33.3,23.6,132.0,104.0,DET,106,0,2016,2015-10-27,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26287,8514,240.0,44.0,90.0,0.489,5.0,28.0,0.179,19.0,22.0,...,17.0,30.3,128.0,125.0,LAL,105,1,2024,2024-04-25,True
26288,8517,240.0,47.0,92.0,0.511,13.0,37.0,0.351,14.0,16.0,...,100.0,34.8,198.0,138.0,CLE,83,0,2024,2024-04-25,True
26289,8515,240.0,44.0,90.0,0.489,5.0,27.0,0.185,12.0,17.0,...,22.2,29.0,172.0,116.0,DEN,112,0,2024,2024-04-25,False
26290,8518,240.0,44.0,90.0,0.489,13.0,30.0,0.433,13.0,19.0,...,34.7,37.1,168.0,130.0,PHI,125,1,2024,2024-04-25,False


#### Ajout de target : bouléen sur l'issue du match suivant

In [35]:
def add_target(group):
    group["target"] = group["won"].shift(-1)
    return group

df = df.groupby("team", group_keys=False).apply(add_target)
df[df["team"] == "WAS"]
df["target"][pd.isnull(df["target"])] = 2
df["target"] = df["target"].astype(int, errors="ignore")
df = df.copy()

C:\Users\rudyv\AppData\Local\Temp\ipykernel_9104\4104344302.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  group["target"] = group["won"].shift(-1)
C:\Users\rudyv\AppData\Local\Temp\ipykernel_9104\4104344302.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  group["target"] = group["won"].shift(-1)
C:\Users\rudyv\AppData\Local\Temp\ipykernel_9104\4104344302.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joinin

#### Préparation pour le machine learning

In [36]:
nulls = pd.isnull(df).sum()
nulls = nulls[nulls > 0]
valid_columns = df.columns[~df.columns.isin(nulls.index)]
df = df[valid_columns].copy()

In [37]:
from sklearn.linear_model import RidgeClassifier
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.model_selection import TimeSeriesSplit

rr = RidgeClassifier(alpha=1)

split = TimeSeriesSplit(n_splits=3)

sfs = SequentialFeatureSelector(rr, 
                                n_features_to_select=30, 
                                direction="forward",
                                cv=split,
                                n_jobs=1
                               )

On défini les colonnes que l'on souhaite conserver pour la suite du projet:

In [38]:
removed_columns = ["season", "date", "won", "target", "team", "team_opp"]
selected_columns = df.columns[~df.columns.isin(removed_columns)]

On défini un `scaler` qui permet de réévaluer chaque statistique par un réel compris entre 0 et 1, ce qui permet un gain de performances pour l'entraînement du modèle.

In [39]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
df[selected_columns] = scaler.fit_transform(df[selected_columns])
df

,Unnamed: 0,mp,fg,fga,fg%,3p,3pa,3p%,ft,fta,...,usg%_max_opp,ortg_max_opp,drtg_max_opp,team_opp,total_opp,home_opp,season,date,won,target
0,0.905239,0.0,0.478261,0.529412,0.377990,0.310345,0.393939,0.356295,0.454545,0.343750,...,0.216667,0.530806,0.488636,NOP,0.276786,0.0,2016,2015-10-27,True,1
1,0.905183,0.0,0.347826,0.338235,0.366029,0.206897,0.212121,0.395487,0.431818,0.421875,...,0.278205,0.554502,0.306818,GSW,0.419643,1.0,2016,2015-10-27,False,0
2,0.951269,0.0,0.391304,0.397059,0.373206,0.241379,0.227273,0.437055,0.363636,0.359375,...,0.089744,0.232227,0.318182,CLE,0.276786,0.0,2016,2015-10-27,True,1
3,0.951213,0.0,0.413043,0.500000,0.322967,0.310345,0.378788,0.368171,0.227273,0.265625,...,0.161538,0.345972,0.306818,CHI,0.294643,1.0,2016,2015-10-27,False,1
4,0.068933,0.0,0.391304,0.323529,0.435407,0.275862,0.348485,0.351544,0.272727,0.234375,...,0.020513,0.203791,0.306818,DET,0.375000,0.0,2016,2015-10-27,False,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26287,0.479095,0.0,0.543478,0.441176,0.526316,0.172414,0.363636,0.212589,0.431818,0.343750,...,0.106410,0.184834,0.545455,LAL,0.366071,1.0,2024,2024-04-25,True,2
26288,0.479264,0.0,0.608696,0.470588,0.578947,0.448276,0.500000,0.416865,0.318182,0.250000,...,0.164103,0.516588,0.693182,CLE,0.169643,0.0,2024,2024-04-25,True,2
26289,0.479151,0.0,0.543478,0.441176,0.526316,0.172414,0.348485,0.219715,0.272727,0.265625,...,0.089744,0.393365,0.443182,DEN,0.428571,0.0,2024,2024-04-25,False,2
26290,0.479320,0.0,0.543478,0.441176,0.526316,0.448276,0.393939,0.514252,0.295455,0.296875,...,0.193590,0.374408,0.602273,PHI,0.544643,1.0,2024,2024-04-25,False,2


#### Prise en compte des tendances sur 10 matchs

On va ici tenter de mettre en évidence les phases d'une saison NBA: une équipe peut être sur une bonne vibe et être très performante pednant une période puis cette tendance peut s'estomper. Pour ce faire, on effectue l'opération de `rolling` qui consiste à boucler sur un nombre de valeurs d'une même colonne (ici 10 matchs d'affilée), puis on calcul la moyenne de ces valeurs. Pour faire cela, on regroupe par team et saison pour éviter les changements éventuels de joueurs d'une saison à l'autre dans une team.

In [40]:
df_rolling = df[list(selected_columns) + ["won", "team", "season"]]

def find_team_averages(team):
    # sélectionne uniquement les colonnes numériques pour calculer la moyenne
    numeric_cols = team.select_dtypes(include=[np.number])
    rolling = numeric_cols.rolling(10).mean()
    # Conservez les colonnes non numériques dans le résultat final
    non_numeric_cols = team.select_dtypes(exclude=[np.number])
    return pd.concat([rolling, non_numeric_cols], axis=1)

df_rolling = df_rolling.groupby(["team", "season"], group_keys=False).apply(find_team_averages)

C:\Users\rudyv\AppData\Local\Temp\ipykernel_9104\3026149532.py:11: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_rolling = df_rolling.groupby(["team", "season"], group_keys=False).apply(find_team_averages)


In [41]:
rolling_cols = [f"{col}_10" for col in df_rolling.columns]
df_rolling.columns = rolling_cols
df = pd.concat([df, df_rolling], axis=1)
df = df.dropna()

#### Ajout des données sur les adversaires

A présent, notre `DataFrame` est quasiment prêt, cependant, nous devons également ajouter pour chaque ligne, les données relatives aux opposants. Pour ce faire, on commence par définir une fonction `shift_col`, qui permet de décaler les données d'une certaines colonne une ligne après et permettra par la suite d'ajouter les informations sur le match suivant à chaque ligne.

In [42]:
def shift_col(dataframe, col_name):
    next_col = dataframe[col_name].shift(-1)
    return next_col

def add_col(df, col_name):
    return df.groupby("team", group_keys=False).apply(lambda x: shift_col(x, col_name))

df["home_next"] = add_col(df, "home")
df["team_opp_next"] = add_col(df, "team_opp")
df["date_next"] = add_col(df, "date")

C:\Users\rudyv\AppData\Local\Temp\ipykernel_9104\711520933.py:6: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  return df.groupby("team", group_keys=False).apply(lambda x: shift_col(x, col_name))
C:\Users\rudyv\AppData\Local\Temp\ipykernel_9104\711520933.py:6: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  return df.groupby("team", group_keys=False).apply(lambda x: shift_col(x, col_name))
C:\Users\rudyv\AppDa

On regroupe les infos sur les opposants pour ajouter cette connaissance à notre algorithme, le tout dans un nouveau `DataFrame` que l'on utilisera pour notre entrâinement de modèle. 

In [43]:
full = df.merge(df[rolling_cols + ["team_opp_next", "date_next", "team"]], left_on=["team", "date_next"], right_on=["team_opp_next", "date_next"])
full

,Unnamed: 0,mp,fg,fga,fg%,3p,3pa,3p%,ft,fta,...,usg%_max_opp_10_y,ortg_max_opp_10_y,drtg_max_opp_10_y,total_opp_10_y,home_opp_10_y,season_10_y,won_10_y,team_10_y,team_opp_next_y,team_y
0,0.968938,0.0,0.456522,0.500000,0.375598,0.379310,0.348485,0.483373,0.454545,0.406250,...,0.274359,0.270616,0.462500,0.286607,0.6,2016.0,True,TOR,SAC,TOR
1,0.289179,0.0,0.326087,0.250000,0.413876,0.310345,0.257576,0.509501,0.522727,0.421875,...,0.126026,0.404739,0.394318,0.398214,0.2,2016.0,True,SAC,TOR,SAC
2,0.718024,0.0,0.326087,0.558824,0.186603,0.206897,0.469697,0.203088,0.159091,0.125000,...,0.157821,0.470142,0.378409,0.405357,0.6,2016.0,False,NOP,DEN,NOP
3,0.356648,0.0,0.413043,0.397059,0.401914,0.137931,0.212121,0.263658,0.431818,0.375000,...,0.124231,0.332227,0.393182,0.343750,0.5,2016.0,True,MIN,ORL,MIN
4,0.340611,0.0,0.391304,0.279412,0.476077,0.241379,0.227273,0.437055,0.454545,0.406250,...,0.202308,0.375355,0.512500,0.320536,0.4,2016.0,True,GSW,LAC,GSW
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30465,0.478364,0.0,0.434783,0.411765,0.416268,0.275862,0.454545,0.279097,0.340909,0.265625,...,0.180128,0.434123,0.547727,0.444643,0.7,2024.0,False,LAL,DEN,LAL
30466,0.478420,0.0,0.369565,0.352941,0.382775,0.448276,0.484848,0.428741,0.363636,0.343750,...,0.170000,0.506161,0.609091,0.403571,0.5,2024.0,True,NYK,PHI,NYK
30467,0.478476,0.0,0.391304,0.455882,0.330144,0.379310,0.439394,0.395487,0.431818,0.359375,...,0.135897,0.326066,0.527273,0.367857,0.5,2024.0,False,PHI,NYK,PHI
30468,0.478195,0.0,0.217391,0.294118,0.224880,0.310345,0.469697,0.305226,0.431818,0.406250,...,0.153846,0.471564,0.486364,0.417857,0.5,2024.0,True,CLE,ORL,CLE


#### Machine learning et test

In [44]:
removed_columns = list(full.columns[full.dtypes == "object"]) + removed_columns
selected_columns = full.columns[~full.columns.isin(removed_columns)]

On entraîne notre modèle avec le set de données complet:

In [45]:
sfs.fit(full[selected_columns], full["target"])

SequentialFeatureSelector(cv=TimeSeriesSplit(gap=0, max_train_size=None, n_splits=3, test_size=None),
                          estimator=RidgeClassifier(alpha=1),
                          n_features_to_select=30, n_jobs=1)

In [49]:
predictors = list(selected_columns[sfs.get_support()])
predictors

['efg%',
 '3par',
 'usg%',
 'ts%_max',
 'usg%_opp',
 'ftr_max_opp',
 'blk%_max_opp',
 'mp_10_x',
 'ts%_10_x',
 'usg%_10_x',
 'drtg_10_x',
 'blk_max_10_x',
 'tov_max_10_x',
 'pts_max_10_x',
 'drtg_max_10_x',
 'mp_opp_10_x',
 'blk_opp_10_x',
 'ast%_opp_10_x',
 'usg%_opp_10_x',
 'fta_max_opp_10_x',
 'won_10_x',
 'drb%_10_y',
 'usg%_10_y',
 'trb%_max_10_y',
 'drtg_max_10_y',
 '3p%_opp_10_y',
 'orb%_opp_10_y',
 'usg%_opp_10_y',
 'drtg_max_opp_10_y',
 'won_10_y']

In [58]:
# on enregistre le model en format pkl pour l'utiliser plus tard
import joblib
joblib.dump(sfs, "model.pkl")
# on enregistre predictors
joblib.dump(predictors, "predictors.pkl")



['predictors.pkl']

On défini la fonction `backtest` comme fonction permettant de réaliser des prédictions à partir de notre modèle, avec:
* _start_ pour définir à partir de combien de saisons de statistiques données, le modèle commence son entraînement (donc il va commencer les prédicitons pour la saison 2017-18 avec les données des deux premières saisons)
* _step_ pour définir le pas que l'on va incrémenter pour délimiter les statistiques données de celles à prédire (les prédictions d'une année se feront donc avec toutes les données sûres des années précédentes) 

In [50]:
def backtest(data, model, predictors, start=2, step=1):
    all_predictions = []
    
    seasons = sorted(data["season"].unique())
    
    for i in range(start, len(seasons), step):
        season = seasons[i]
        train = data[data["season"] < season]
        test = data[data["season"] == season]
        
        model.fit(train[predictors], train["target"])
        
        preds = model.predict(test[predictors])
        preds = pd.Series(preds, index=test.index)
        combined = pd.concat([test["target"], preds], axis=1)
        combined.columns = ["actual", "prediction"]
        
        all_predictions.append(combined)
    return pd.concat(all_predictions)

In [51]:
predictions = backtest(full, rr, predictors)

#### Prédiction naïve et évaluation du modèle

Pour déterminer une limite de confiance pour la précision, nous allons partir du constat suivant: en NBA les équipes gagnent plus souvent à domicile qu'à l'extérieur. Nous allons donc regarder quel est le pourcentage de victoires à domicile et à l'extérieur.

In [52]:
df.groupby(["home"]).apply(lambda x: x[x["won"] == 1].shape[0] / x.shape[0])

C:\Users\rudyv\AppData\Local\Temp\ipykernel_9104\330557953.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df.groupby(["home"]).apply(lambda x: x[x["won"] == 1].shape[0] / x.shape[0])


home
0.0    0.429386
1.0    0.570028
dtype: float64

On en déduit que si on prédit uniquement que la team à domicile va gagner, on aura 57% de probabilité que cela soit vrai.

#### Validation du modèle

In [53]:
from sklearn.metrics import accuracy_score

accuracy_score(predictions["actual"], predictions["prediction"])

0.6484013820623875

On obtient donc 65% de prédictions correctes, ce qui est mieux que si on se fie seulement au fait que l'équipe joue à domicile.

Pour améliorer notre modèle, on peut changer notre modèle pour un modèle plus puissant comme XG Boost ou Random Forest classifier.
___